In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:13 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-25 00:05:56--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-04-25 00:05:57 (6.58 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Furniture_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Furniture_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24509695|R3VR960AHLFKDV|B004HB5E0E|     488241329|Shoal Creek Compu...|       Furniture|          4|            0|          0|   N|                Y|... desk is very ...|This desk is very...| 2015-08-31|
|         US|   34731776|R16LGVMFKIUT0G|B0042TNMMS|     205864445|Dorel Home Produc...|       Furniture|          5|    

In [5]:
cleaned_df = df.dropna()
cleaned_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24509695|R3VR960AHLFKDV|B004HB5E0E|     488241329|Shoal Creek Compu...|       Furniture|          4|            0|          0|   N|                Y|... desk is very ...|This desk is very...| 2015-08-31|
|         US|   34731776|R16LGVMFKIUT0G|B0042TNMMS|     205864445|Dorel Home Produc...|       Furniture|          5|    

### Create DataFrames to match tables

In [7]:
# Create the vine_table. DataFrame
vine_df = cleaned_df.select(["review_id","star_rating","helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3VR960AHLFKDV|          4|            0|          0|   N|                Y|
|R16LGVMFKIUT0G|          5|            0|          0|   N|                Y|
|R1AIMEEPYHMOE4|          5|            1|          1|   N|                Y|
|R1892CCSZWZ9SR|          3|            0|          0|   N|                Y|
|R285P679YWVKD1|          3|            0|          0|   N|                N|
| RLB33HJBXHZHU|          5|            0|          0|   N|                Y|
|R1VGTZ94DBAD6A|          5|            2|          2|   N|                Y|
|R168KF82ICSOHD|          5|            0|          0|   N|                Y|
|R20DIYIJ0OCMOG|          5|            0|          0|   N|                Y|
| RD46RNVOHNZSC|          5|            0|          0|   N|     

### Deliverable 2

In [8]:
# 1. Filter the data and create a new DataFrame or table to retrieve all the rows 
#where the total_votes count is equal to or greater than 20 to pick reviews that 
#are more likely to be helpful and to avoid having division by zero errors later on.
from pyspark.sql.functions import col
total_votes_df = vine_df.filter(col("total_votes") >= 20)
total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RL8D0KJ0J9L0O|          5|          152|        165|   N|                Y|
|R1BEINAIQFBRJC|          5|           21|         23|   N|                Y|
|R2L59KIJH302P9|          4|           26|         26|   N|                Y|
| RR99CPG695T0I|          5|          215|        248|   N|                N|
|R1XQNKKUPCMWVO|          5|           43|         44|   N|                Y|
|R3JUXVCT1NSK2A|          3|           25|         26|   N|                Y|
|R3GNSIFV1J2Y2B|          1|           15|         60|   N|                N|
| RTCRZARYY4LXX|          5|           52|         54|   N|                Y|
|R3OFB4P7Y8WR27|          1|           15|         26|   N|                Y|
|R3MTAYGQM25N63|          4|           58|         59|   N|     

In [9]:
# 2. Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
helpful_votes_df = total_votes_df.withColumn('percent_votes',col('helpful_votes')/col('total_votes')).alias('percent_votes').filter(col("percent_votes") >= 0.5)
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| RL8D0KJ0J9L0O|          5|          152|        165|   N|                Y|0.9212121212121213|
|R1BEINAIQFBRJC|          5|           21|         23|   N|                Y|0.9130434782608695|
|R2L59KIJH302P9|          4|           26|         26|   N|                Y|               1.0|
| RR99CPG695T0I|          5|          215|        248|   N|                N|0.8669354838709677|
|R1XQNKKUPCMWVO|          5|           43|         44|   N|                Y|0.9772727272727273|
|R3JUXVCT1NSK2A|          3|           25|         26|   N|                Y|0.9615384615384616|
| RTCRZARYY4LXX|          5|           52|         54|   N|                Y|0.9629629629629629|
|R3OFB4P7Y8WR27|          1|  

In [11]:
# 3. Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid)
paid_df = helpful_votes_df.filter(col("vine") == "Y").show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2BQOD1R0228FN|          3|           17|         26|   Y|                N|0.6538461538461539|
| RC31RUPFOHBHQ|          4|          102|        117|   Y|                N|0.8717948717948718|
| REN3N1WITLF1Y|          5|           33|         37|   Y|                N|0.8918918918918919|
| R71RZQ9UZVG47|          4|           38|         47|   Y|                N|0.8085106382978723|
|R38NMQBH88HLM6|          4|           18|         24|   Y|                N|              0.75|
|R33FGX9EE3QVR6|          4|           26|         26|   Y|                N|               1.0|
|R1KIOIK6WEYE59|          3|           19|         20|   Y|                N|              0.95|
|R25X9BMOB3FD0E|          4|  

In [12]:
# 4. Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid)
non_paid_df = helpful_votes_df.filter(col("vine") == "N").show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| RL8D0KJ0J9L0O|          5|          152|        165|   N|                Y|0.9212121212121213|
|R1BEINAIQFBRJC|          5|           21|         23|   N|                Y|0.9130434782608695|
|R2L59KIJH302P9|          4|           26|         26|   N|                Y|               1.0|
| RR99CPG695T0I|          5|          215|        248|   N|                N|0.8669354838709677|
|R1XQNKKUPCMWVO|          5|           43|         44|   N|                Y|0.9772727272727273|
|R3JUXVCT1NSK2A|          3|           25|         26|   N|                Y|0.9615384615384616|
| RTCRZARYY4LXX|          5|           52|         54|   N|                Y|0.9629629629629629|
|R3OFB4P7Y8WR27|          1|  

In [14]:
#5. Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).
from pyspark.sql.functions import col,when,count,lit
ratings_total_df = helpful_votes_df.groupBy("vine").agg(
    count(col("vine")).alias("Total_Reviews"),
    count(when(col("star_rating") == 5, True)).alias("Total_5_Star_Reviews"),
    (count(when(col("star_rating") == 5, True))/count(col("vine"))*100).alias("%_5_Star_To_Total")).show()

+----+-------------+--------------------+-----------------+
|vine|Total_Reviews|Total_5_Star_Reviews|%_5_Star_To_Total|
+----+-------------+--------------------+-----------------+
|   Y|          136|                  74|54.41176470588235|
|   N|        18002|                8472|47.06143761804244|
+----+-------------+--------------------+-----------------+

